# Part 1
Each point is listed by an x,y coordinates. \
Each pipe is listed left to right or top to bottom coordinates

In [142]:
def get_pipes(positions, map_dict, skip_ground=True):
    pipes = []
    
    for idx,line in enumerate(positions):
        y = 2*idx + 1
        for left, p in enumerate(line):
            x= 2*left+1
            # Middle of tile
            point = [x,y]
            if p=='S':
                start = point
            elif skip_ground and p=='.':
                continue
            else:
                ## Find the pipe coordinates
                pipe = [[j[0]+point[0], j[1]+point[1]] for i, j in enumerate(map_dict[p])]
                pipes += [pipe]
    #Look at the start point to figure the type of pipe it is
    start =  [[j[0]+start[0], j[1]+start[1]] for i, j in enumerate(map_dict['|'])]
    
    return pipes, start

def get_loop(pipes, start):
    ## Unpacks each coordinate for all pipes
    all_pipes =[x for l in pipes for x in l]  
    all_pipes+= start
    ## Find connection points
    connected = [i for i in all_pipes if all_pipes.count(i)>1] 
    ## Find pipes that are connected on both ends
    conn_pipes = [i for i in pipes if i[0] in connected and i[1] in connected]
    
    # Find pipes that are in the loop
    end = start[0]
    loop = [end]
    curr_pipe = conn_pipes
    ## While we have not reached the other point of the pipe
    while end!= start[1]:
        ## Find the adjoining pipe
        con= [i for i in curr_pipe if (i[0]==end) or i[1]==end][0]
        ## Remove adjoining pipe from the search list
        curr_pipe.remove(con) 
        ## Find the next connection point
        end=[i for i in con if i!= end][0]
        loop+=[end]
    return loop

In [140]:
# Map of the cahnges of coordinate from the midpoint of each 2*2 block that a pipe lays on
map_dict = {'|':[[ 0,-1],[ 0, 1]],
            '-':[[-1, 0],[ 1, 0]], 
            'L':[[ 0,-1],[ 1, 0]],
            'J':[[-1, 0],[ 0,-1]], 
            '7':[[-1, 0],[ 0, 1]], 
            'F':[[ 0, 1],[ 1, 0]]}

In [141]:
with open('day10.txt') as f:
    text = f.read()
positions = [[a for a in i] for i in text.split('\n')]

pipes, start = get_pipes(positions, map_dict)
loop = get_loop(pipes, start)
int(len(loop)/2)

6701

# Part 2

In [148]:
from shapely.geometry import Polygon, Point

# Tile mappings with '.'
tile_map = {**map_dict, **{'.':[[0,0], [0,0]]}}

### Same as Part 1 except the mapping is different
with open('day10.txt') as f:
    text = f.read()
    
positions = [[a for a in i] for i in text.split('\n')]
pipes, start = get_pipes(positions, tile_map, skip_ground=False)
loop = get_loop(pipes, start)

# Get tiles that are not in the loop
tiles = [i for i in pipes if i[0] not in loop and i[1] not in loop]
# Converting pipes to points (midpoint of each tile)
points = [list(map(lambda x: int(x/2), [i[0][0]+i[1][0], i[0][1]+i[1][1]])) for i in tiles]
count=0
## Get enclosed loop and initiate in shapely
poly = loop + [loop[0]]
polygon = Polygon(poly)

## Find if each point/tile is enclosed in the polygon
for point in points:
    if polygon.contains(Point(point)):
        count+=1
print(count)

303
